Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms,query, constantVariables
import cdapython
print(cdapython.__file__)
print(cdapython.__version__)
integration_host = "http://35.192.60.10:8080/"
Q.set_host_url(integration_host)

/Users/amanda.charbonneau/github/cda-python/cdapython/__init__.py
2022.5.23


CDA data comes from three sources:
- The [Proteomic Data Commons](https://proteomic.datacommons.cancer.gov/pdc/) (PDC)
- The [Genomic Data Commons](https://gdc.cancer.gov/) (GDC)
- The [Imaging Data Commons](https://datacommons.cancer.gov/repository/imaging-data-commons) (IDC)

The CDA makes this data searchable in four main endpoints:

- `subject`: Specific, unique, individuals
- `researchsubject`: Study-individual aggregate entities. A Subject who was part of three studies will appear as three ResearchSubjects
- `specimen`: Samples taken from individual
- `file`: Data about Subjects, ResearchSubjects, Specimens, and their associated information

and two endpoints that offer deeper information about data in the researchsubject endpoint:

- `diagnosis`: Information about what medical diagnosis a researchsubject has
- `treatment`: Information about what medical treatment(s) were performed for a given diagnosis

If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specfic format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `file`.

In CDA search, these concepts can also be chained together, so you can look specifically for specimen subjects, or researchsubject diagnoses. In the four 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in specimen files. Diagnosis and treatment do not have files directly associated with them and so can only be used to find files in conjunction with the other searches.

In all cases, any search can use any metadata field, the only difference between search types is what type of data you return by default. 



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [2]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


<div style="background-color:#6ce6b9;color:black;padding:20px;">
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what Subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [3]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 3748 ms


We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [4]:
subjectresults


            QueryID: 6b1f5113-18d4-40f0-9a49-4062a5653fea
            
            Offset: 0
            Count: 100
            Total Row Count: 2314
            More pages: True
            

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. Then it tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
Now that we've seen the metadata about our results, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [5]:
subjectresults.to_dataframe()

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
0,ACRIN-FMISO-Brain-010,"[{'system': 'IDC', 'value': 'ACRIN-FMISO-Brain...",Homo sapiens,None,None,None,None,[acrin_fmiso_brain],None,None,None
1,C140343,"[{'system': 'PDC', 'value': 'C140343'}]",Homo sapiens,male,white,not hispanic or latino,None,[Proteogenomic Analysis of Pediatric Brain Can...,Alive,None,Not Reported
2,C3L-01327,"[{'system': 'GDC', 'value': 'C3L-01327'}, {'sy...",Homo sapiens,male,white,not hispanic or latino,-27160,"[cptac_gbm, CPTAC3-Discovery, CPTAC-3]",Dead,646,Cancer Related
3,C808110,"[{'system': 'PDC', 'value': 'C808110'}]",Homo sapiens,female,white,not hispanic or latino,None,[Proteogenomic Analysis of Pediatric Brain Can...,Alive,None,Not Reported
4,GENIE-GRCC-e6a259ca,"[{'system': 'GDC', 'value': 'GENIE-GRCC-e6a259...",Homo sapiens,female,not allowed to collect,not allowed to collect,-17166,[GENIE-GRCC],Not Reported,None,None
...,...,...,...,...,...,...,...,...,...,...,...
95,QIN-GBM-TR-01,"[{'system': 'IDC', 'value': 'QIN-GBM-TR-01'}]",Homo sapiens,None,None,None,None,[qin_gbm_treatment_response],None,None,None
96,RIDER Neuro MRI-2579032163,"[{'system': 'IDC', 'value': 'RIDER Neuro MRI-2...",Homo sapiens,None,None,None,None,[rider_neuro_mri],None,None,None
97,TCGA-02-0336,"[{'system': 'GDC', 'value': 'TCGA-02-0336'}]",Homo sapiens,None,None,None,None,[TCGA-GBM],None,None,None
98,TCGA-02-0446,"[{'system': 'GDC', 'value': 'TCGA-02-0446'}]",Homo sapiens,male,white,not hispanic or latino,-22541,[TCGA-GBM],Dead,282,None


By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any pandas functions to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs. We're also telling it we want the word "black" regardless of capitalization with `case=False`:


In [6]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", case=False, na=False)]

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
38,TCGA-06-A7TL,"[{'system': 'GDC', 'value': 'TCGA-06-A7TL'}]",Homo sapiens,female,black or african american,not hispanic or latino,-11297,[TCGA-GBM],Alive,None,None
69,TCGA-26-A7UX,"[{'system': 'GDC', 'value': 'TCGA-26-A7UX'}]",Homo sapiens,male,black or african american,not hispanic or latino,-17365,[TCGA-GBM],Alive,None,None


There are three subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the [Pagination](../Pagination.ipynb) notebook. Or, learn how to get summary information from search results in the [Data Summaries](../DataSummaries.ipynb) notebook.


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A subject is a specific, unique individual: for e.g. a single human. When consent allows, a given entity will have a single subject ID that can be connected to all their studies and data across all datasets</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the subject had there</li>
  <li><b>species:</b> The species of the subject</li>
  <li><b>sex:</b> A reference to the biological sex of the donor organism. </li>
  <li><b>race:</b> The race of the subject</li>
  <li><b>ethnicity:</b> The ethnicity of the subject</li>
  <li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days</li>
  <li><b>subject_associated_project:</b> An embedded array of the names of projects (studies) the subject was part of</li>
  <li><b>vital_status:</b> Whether the subject is alive</li>
  <li><b>age_at_death:</b> The number of days after first enrollment that the subject died</li>
  <li><b>cause_of_death:</b> The cause of death, if known</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our critera, we can also run our query against the researchsubject endpoint:

In [7]:
researchsubjectresults = myquery.researchsubject.run()
researchsubjectresults

Getting results from database

Total execution time: 3490 ms



            QueryID: 35ba8bad-9d0d-4fb5-9b9d-c2445caf707d
            
            Offset: 0
            Count: 100
            Total Row Count: 2923
            More pages: True
            

Now we see that our 2314 subjects have 2923 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [8]:
researchsubjectresults.to_dataframe()

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,104c3a87-2139-11ea-aee1-0e1aae319e49,"[{'system': 'PDC', 'value': '104c3a87-2139-11e...",CPTAC3-Discovery,Glioblastoma,Brain,C3L-03407
1,153c2442-ea61-4b5e-8c5c-c71d287c6055,"[{'system': 'GDC', 'value': '153c2442-ea61-4b5...",TCGA-GBM,Gliomas,Brain,TCGA-28-5209
2,1edfe67c-1142-4498-927c-9b7b426af7a3,"[{'system': 'GDC', 'value': '1edfe67c-1142-449...",TCGA-GBM,Gliomas,Brain,TCGA-02-0339
3,3d58ded1-b7cf-420c-ba59-5b424683f0df,"[{'system': 'GDC', 'value': '3d58ded1-b7cf-420...",TCGA-LGG,Gliomas,Brain,TCGA-HT-7686
4,559a8f6f-dd60-43b7-ae09-16cc4206064c,"[{'system': 'GDC', 'value': '559a8f6f-dd60-43b...",TCGA-LGG,Gliomas,Brain,TCGA-HT-A74L
...,...,...,...,...,...,...
95,723cad09-aa2d-4492-b19d-661b310e1c58,"[{'system': 'GDC', 'value': '723cad09-aa2d-449...",TCGA-LGG,Gliomas,Brain,TCGA-HT-A74O
96,7338e476-10ff-444a-a16f-429d28355f65,"[{'system': 'GDC', 'value': '7338e476-10ff-444...",TCGA-GBM,Gliomas,Brain,TCGA-28-5213
97,834e560a-a506-456d-baf2-a206ca058153,"[{'system': 'GDC', 'value': '834e560a-a506-456...",TCGA-LGG,Gliomas,Brain,TCGA-DU-7010
98,ACRIN-DSC-MR-Brain-037__acrin_dsc_mr_brain,"[{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...",acrin_dsc_mr_brain,None,Brain,ACRIN-DSC-MR-Brain-037


Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered in the [Merging Results](../MergingResults.ipynb) notebook.


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a research study, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subject’s privacy. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
  <li><b>id:</b> The unique identifier for this researchsubject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the researchsubject had there</li>
  <li><b>member_of_research_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>primary_diagnosis_condition:</b> The cancer, disease or other condition under study</li>
  <li><b>primary_diagnosis_site:</b> The primary_disease_site that qualifies the researchsubject for the research_project</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [9]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 3452 ms


,id,identifier,primary_diagnosis,age_at_diagnosis,morphology,stage,grade,method_of_diagnosis,subject_id,researchsubject_id
0,02ce9b1f-18f4-45ec-a487-63f9c74bb422,"[{'system': 'GDC', 'value': '02ce9b1f-18f4-45e...",Mixed germ cell tumor,None,9085/3,None,Not Reported,None,GENIE-MSK-P-0002849,3ecb27b2-75cc-44d0-879e-035e19f39193
1,0f7bb10c-a17e-4a98-b559-461edbf840c8,"[{'system': 'GDC', 'value': '0f7bb10c-a17e-4a9...",Glioblastoma,23121,9440/3,None,Not Reported,None,C3N-01366,a1da2f64-5a24-4e6b-abc1-e70aae23082c
2,13c464b8-bd62-5d47-91ff-4b2c9183c48b,"[{'system': 'GDC', 'value': '13c464b8-bd62-5d4...","Oligodendroglioma, NOS",12764,9450/3,None,not reported,None,TCGA-DU-5870,eb42625e-fbbb-4bee-bbe9-315b3ca2138a
3,195c5853-a955-551b-8601-6f0d8a08cea8,"[{'system': 'GDC', 'value': '195c5853-a955-551...",Glioblastoma,14838,9440/3,None,not reported,None,TCGA-08-0351,91063e7e-60db-47a5-aeca-af7c37b1074e
4,2faa8b42-f38e-50f2-9920-cafeafc989ae,"[{'system': 'GDC', 'value': '2faa8b42-f38e-50f...",Glioblastoma,28696,9440/3,None,not reported,None,TCGA-02-0281,42fbe95f-9068-45a8-a8a2-c24db757eae6
...,...,...,...,...,...,...,...,...,...,...
95,7a3112c5-a878-5b23-bdbc-3aa91d2cede6,"[{'system': 'GDC', 'value': '7a3112c5-a878-5b2...",Mixed glioma,20128,9382/3,None,not reported,None,TCGA-P5-A77X,878ced76-3144-4141-896f-50e3e863df36
96,7b8b418c-70eb-5b2c-a880-1c869f324f21,"[{'system': 'GDC', 'value': '7b8b418c-70eb-5b2...",Glioblastoma,22332,9440/3,None,not reported,None,TCGA-06-1800,b1ce95bf-7441-4aac-8131-19a177d2b8ac
97,8e43bc6d-48f0-5567-8333-64c91a50d4eb,"[{'system': 'GDC', 'value': '8e43bc6d-48f0-556...","Astrocytoma, anaplastic",16300,9401/3,None,not reported,None,TCGA-P5-A736,ac3779f6-cc8b-428b-9531-03562a614515
98,91404a49-c46a-43c3-8bdc-6ef7e4feba82,"[{'system': 'GDC', 'value': '91404a49-c46a-43c...",Glioblastoma,25423,9440/3,None,Not Reported,None,C3N-01857,9b84c1a2-d24d-4085-b597-eff3d2203756



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A diagnosis is a medical classification of a disease for a given research subject in a given study. A single research subject may have different diagnoses across different studies</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this diagnosis in this research subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the diagnosed researchsubject had there</li>
  <li><b>primary_diagnosis:</b> The main medical diagnosis for this subject in this study</li>
  <li><b>age_at_diagnosis:</b> The subjects age in days after birth on the day they were first diagnosed</li>
  <li><b>morphology:</b> The <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> diagnosic code for this diagnosis</li>
  <li><b>stage:</b> A measure of disease spread. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> A measure of cell abnormality. Different diseases may use different grading criteria, please refer to the originating data source to see what grading system is reported</li>
  <li><b>method_of_diagnosis:</b> The test or system used for determining the diagnosis</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on research subjects that have a given diagnosis that meets our search criteria:

In [10]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 3535 ms


,id,identifier,treatment_type,treatment_outcome,days_to_treatment_start,days_to_treatment_end,therapeutic_agent,treatment_anatomic_site,treatment_effect,treatment_end_reason,number_of_cycles,subject_id,researchsubject_id,researchsubject_diagnosis_id
0,0611032f-6cde-5d42-b305-23219b4b486a,"[{'system': 'GDC', 'value': '0611032f-6cde-5d4...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-08-0346,5536201c-e739-46e6-8200-0bdcc28ae9ef,202921e5-a4e8-5d23-9d3c-de53394f93a9
1,08cabbfd-c1c2-5689-a1e6-ae03f0e8ecc9,"[{'system': 'GDC', 'value': '08cabbfd-c1c2-568...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-RY-A845,36615695-921c-4c8a-93d6-6c788079beae,cbf0dbb0-8f9c-549d-b766-23bf00479cc7
2,1e45d588-c532-5952-b8e5-56cc67267853,"[{'system': 'GDC', 'value': '1e45d588-c532-595...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-DU-8158,b8df1577-2a75-4eb7-ab27-f2dd76160730,7481b252-5efc-5ed9-9d2a-b049302a3869
3,2412b555-02be-5e02-8c26-cc4db13cb11c,"[{'system': 'GDC', 'value': '2412b555-02be-5e0...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-HT-7477,38f4b9f3-6709-40ff-9a81-5fa2111b1ce6,f15ab57f-5079-5386-9a7b-9dfb1a8f2e1c
4,2afa81c1-efba-5fcd-b171-74820ac8d991,"[{'system': 'GDC', 'value': '2afa81c1-efba-5fc...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-32-2615,30011f30-2926-47f2-a8f3-4f1b58ffa227,c57d8c00-a082-56c0-9d5e-389441cc9523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,34b6e7f0-6cca-5054-90a0-fb65073ebca9,"[{'system': 'GDC', 'value': '34b6e7f0-6cca-505...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-WH-A86K,42156c6a-416e-4832-9c95-cee6084b0910,42519c28-1489-53d6-8dc6-907f0b377f71
96,3c9a6945-d2a1-526c-aab5-edd8ecf3152e,"[{'system': 'GDC', 'value': '3c9a6945-d2a1-526...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-HT-7879,c916b9a4-008e-4d79-a50c-f15c3549fcf6,616d6f27-c72a-59db-b7fd-a2f7962ee604
97,4440d709-1beb-5233-9781-ae767e91b0de,"[{'system': 'GDC', 'value': '4440d709-1beb-523...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-12-1598,2b90648c-c99e-4d4d-8d9b-67ff2c4e6115,62f86760-df1c-5925-8b1a-e13d623a0183
98,48f9d02d-5dfe-5124-85d9-ec9fdd7d8f77,"[{'system': 'GDC', 'value': '48f9d02d-5dfe-512...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-06-1084,ac3582a9-46b1-4322-95fa-9e8b149b8044,d3175d4f-7edc-574e-8682-1a5a86427514



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i>A treatment is a medical intervention for a diagnosed disease in a given subject in a given study. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The unique identifier for this treatment of this diagnosis in this research subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the treated researchsubject had there</li>
  <li><b>treatment_type:</b> The medical intervention undertaken</li>
  <li><b>treatment_outcome:</b> The result of the medical intervention</li>
  <li><b>days_to_treatment_start:</b> </li>
  <li><b>days_to_treatment_end:</b> </li>
  <li><b>therapeutic_agent:</b> What treatment or drug was used for this researchsubject</li>
  <li><b>treatment_anatomic_site:</b> The specific body location of the treatment</li>
  <li><b>treatment_effect:</b> </li>
  <li><b>treatment_end_reason:</b> </li>
  <li><b>number_of_cycles:</b> </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimens

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [11]:
specresults =  myquery.specimen.run()
print(specresults)

Getting results from database

Total execution time: 3511 ms

            QueryID: e348a333-9d2a-419c-99b4-bb4fd44b7ce2
            
            Offset: 0
            Count: 100
            Total Row Count: 39150
            More pages: True
            


Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as seperate speciments. Let's look at a few:

In [12]:
specresults.to_dataframe()

,id,identifier,associated_project,age_at_collection,primary_disease_type,anatomical_site,source_material_type,specimen_type,derived_from_specimen,subject_id,researchsubject_id
0,018966a4-712b-4921-a7e7-02e390f43288,"[{'system': 'GDC', 'value': '018966a4-712b-492...",TCGA-GBM,-14729,Gliomas,None,Blood Derived Normal,analyte,790f03cf-ad8f-4cde-8ba5-f465837c687f,TCGA-06-0139,5bbeb8a7-3ac2-4ef1-bd48-6f83bc994a75
1,0190d8c0-20ab-5475-8804-c99979e77dbe,"[{'system': 'GDC', 'value': '0190d8c0-20ab-547...",TCGA-LGG,-14716,Gliomas,None,Primary Tumor,portion,5665435a-9b77-4504-a60a-b6d3edeb341f,TCGA-R8-A6MK,65836b59-b469-4b89-adf5-e65735e494f5
2,01ed9fd7-7d44-401b-a6b7-4fc6922813fd,"[{'system': 'GDC', 'value': '01ed9fd7-7d44-401...",TCGA-GBM,-26247,Gliomas,None,Primary Tumor,slide,37af7534-d545-45ee-9de3-387ad85d98f7,TCGA-06-0644,652a39f0-c539-4303-bedc-f0d7f4669d9f
3,02117459-65da-4a3c-ba9a-ead2c0daa5ba,"[{'system': 'GDC', 'value': '02117459-65da-4a3...",TCGA-LGG,-17501,Gliomas,None,Blood Derived Normal,portion,fa252999-283a-4ddc-84b5-17257939cc73,TCGA-RY-A840,738143b3-2d82-42f0-9fd4-8cbabc9c9b4f
4,023d5905-9dbf-40a7-90c1-761d3f48ad9a,"[{'system': 'GDC', 'value': '023d5905-9dbf-40a...",TCGA-GBM,-22417,Gliomas,None,Blood Derived Normal,analyte,19496c06-447b-437a-bae8-20d1b0c411c5,TCGA-02-0046,5b8f05fa-0145-4142-8d41-5980e4d36d81
...,...,...,...,...,...,...,...,...,...,...,...
95,485cb0f3-e668-47a5-8b8a-0f8f81688ef2,"[{'system': 'GDC', 'value': '485cb0f3-e668-47a...",TCGA-GBM,-21768,Gliomas,None,Primary Tumor,aliquot,6b524320-94c8-4035-b1d1-e1d3d3fe8679,TCGA-12-5301,10efeb3c-ef55-4227-91b9-8c2bb4ff43d5
96,49280503-5136-4c10-8e42-061abbb6fa05,"[{'system': 'GDC', 'value': '49280503-5136-4c1...",TCGA-GBM,-27111,Gliomas,None,Primary Tumor,aliquot,3b705faf-6584-4dd8-b5f5-423ec3ee7e60,TCGA-26-5134,20bad0d5-3135-49a4-a119-fa7a1e56fd1b
97,49289dc2-31a6-427d-a7b9-e8e3613e17c1,"[{'system': 'GDC', 'value': '49289dc2-31a6-427...",TCGA-LGG,-16280,Gliomas,None,Primary Tumor,aliquot,d1469517-114e-41da-9b50-ccecea62826b,TCGA-HT-7884,86aa5790-9afd-4653-926a-625516df1f56
98,49ae4c42-847f-4da6-9a41-b9eaaa8831f1,"[{'system': 'GDC', 'value': '49ae4c42-847f-4da...",TCGA-LGG,-17432,Gliomas,None,Blood Derived Normal,aliquot,c55220e7-4667-42a2-a8de-4641a29cae1d,TCGA-EZ-7264,fc3b7596-f515-446f-81db-fed0154ca2c5



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this specimen</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the specimen had there</li>
  <li><b>associated_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>age_at_collection:</b> The subjects age in days (counting backwards to birth) on the day of the collection of the proximate specimen</li>
  <li><b>primary_disease_type:</b> The disease that qualifies the researchsubject for the associated_project</li>
  <li><b>anatomical_site:</b> The body part from which the proximate specimen was taken</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide</li>
  <li><b>derived_from_specimen:</b> For derived samples, the `id` for the original sample</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens: 

In [13]:
myquery.file.run()

Getting results from database

Total execution time: 3522 ms



            QueryID: 1aa12ab0-ffe0-4588-8514-51c3960d8d5c
            
            Offset: 0
            Count: 100
            Total Row Count: 4530800
            More pages: True
            

In [14]:
fileresults = myquery.file.run()
fileresults.to_dataframe()

Getting results from database

Total execution time: 3805 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,researchsubject_specimen_id,researchsubject_id,subject_id
0,1b450a73-a16d-4de4-b140-9295335063ad,"[{'system': 'PDC', 'value': '1b450a73-a16d-4de...",NCI-11Plex-2-F12-f10321.psm,Peptide Spectral Matches,Text,tsv,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:1b450a73-a16d-4de4-b140-92953350...,2850423,5e82ca67c560b6ab0b175ee5057ca02e,Proteomic,None,None,,,C952635
1,ef900de0-892c-4e2e-8eae-74caf55de54c,"[{'system': 'GDC', 'value': 'ef900de0-892c-4e2...",TCGA-LGG.c6bdb34c-fa93-4736-a842-afec0d9fa990....,Structural Variation,Transcript Fusion,TSV,TCGA-LGG,drs://dg.4DFC:ef900de0-892c-4e2e-8eae-74caf55d...,15444,293a480b7106cc7945f5807f9eb961de,Genomic,None,None,,,TCGA-DH-5140
2,d43ef537-6cd0-446a-9bf0-fe0767ed6bcf,"[{'system': 'GDC', 'value': 'd43ef537-6cd0-446...",TCGA-R8-A6YH-01A-02-TS2.AD2BC93E-924F-46FC-A41...,Biospecimen,Slide Image,SVS,TCGA-LGG,drs://dg.4DFC:d43ef537-6cd0-446a-9bf0-fe0767ed...,356728009,78c73f88728bf3c73fb148105eb9348f,Genomic,None,None,,,TCGA-R8-A6YH
3,58581feb-03c1-496c-9328-e8dda42c0d59,"[{'system': 'GDC', 'value': '58581feb-03c1-496...",5c6ea055-2352-4111-9f8d-6eec3b9b3847.methylati...,DNA Methylation,Methylation Beta Value,TXT,CPTAC-3,drs://dg.4DFC:58581feb-03c1-496c-9328-e8dda42c...,22617382,617b0a0fd0e824959643edd0a2fc75ff,Genomic,None,phs001287,,,C3N-01334
4,1fd6817e-fa11-4569-a740-86584eaff394,"[{'system': 'PDC', 'value': '1fd6817e-fa11-456...",10CPTAC_GBM_W_PNNL_20190501_B3S2_f09.psm,Peptide Spectral Matches,Text,tsv,CPTAC3-Discovery,drs://dg.4DFC:1fd6817e-fa11-4569-a740-86584eaf...,4543929,11ab9596db36bd898520de88b4d7ad84,Proteomic,None,None,,,C3N-01368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,70b44166-c646-43eb-8d76-b567bf9328ac,"[{'system': 'PDC', 'value': '70b44166-c646-43e...",NCI-11pex-6C-Phos-c05377.mzML.gz,Processed Mass Spectra,Open Standard,mzML,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:70b44166-c646-43eb-8d76-b567bf93...,220183305,756391f44696965c00f4902a5bdcf2ea,Proteomic,None,None,,,C311067
96,da7f4908-e98d-4ce6-93e3-a721276f733b,"[{'system': 'GDC', 'value': 'da7f4908-e98d-4ce...",179f277a-8e72-4008-8a27-ea6e0d4ec7db.wxs.aliqu...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-GBM,drs://dg.4DFC:da7f4908-e98d-4ce6-93e3-a721276f...,965606,e18a3fce6c7fc6bdc2d09858cc17bd1e,Genomic,None,None,,,TCGA-19-1787
97,8341914b-205d-4765-91ad-3e223c465459,"[{'system': 'PDC', 'value': '8341914b-205d-476...",10CPTAC_GBM_W_PNNL_20190501_B3S2_f24.mzid.gz,Peptide Spectral Matches,Open Standard,mzIdentML,CPTAC3-Discovery,drs://dg.4DFC:8341914b-205d-4765-91ad-3e223c46...,8766142,af764efe01d6ff020fcd8a9af690aed7,Proteomic,None,None,,,C3N-03184
98,ade18349-cd37-4da0-94c2-2a9e664002ac,"[{'system': 'GDC', 'value': 'ade18349-cd37-4da...",962fbeae-2f42-4969-b35d-b1ec852c57ca.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,MAF,TCGA-GBM,drs://dg.4DFC:ade18349-cd37-4da0-94c2-2a9e6640...,23065,ed693ebc68c8af3004ca47a36a1b4f4d,Genomic,None,None,,,TCGA-32-4211


As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial [Chaining Queries](../ChainingQueries): Combine information from multiple endpoints, and build And/Or/Like and other advanced query strings.

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the [Data Summaries tutorial](../DataSummaries) to try it.



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this file</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the file had there</li>
  <li><b>label:</b> The full name of the file</li>
  <li><b>data_catagory:</b> A desecription of the kind of general kind data the file holds</li>
  <li><b>data_type:</b> A more specific descripton of the data type</li>
  <li><b>file_format:</b> String to identify the full file extension including compression extensions</li>
  <li><b>associated_project:</b> The name the data center uses for the study this file was generated for</li>
  <li><b>drs_uri:</b> A unique identifier that can be used to retreive this specific file from a server</li>
  <li><b>byte_size:</b> Size of the file in bytes</li>
  <li><b>checksum:</b> The md5 value for the file</li>
  <li><b>data_modality:</b> Describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging"</li>
  <li><b>imaging_modality:</b> For files with the `data_modality` of "Imaging", a descriptor for the image type</li>
  <li><b>dbgap_accession_number:</b> The project id number for this data on dbGaP</li>
</ul>  

</div>
    
---
